In [4]:
import os
import cPickle as pickle
import numpy as np
from utils import init_log, score
from sk_models.logistic_regressor import LogisticRegressor
from sk_models.random_forest_classifier import RandomForest
from sk_models.bagging_classifier import Bagging
from sk_models.adaboost_classifier import AdaBoost
from sk_models.mlp_classifier import MLP
from sk_models.sk_model import SKModel
from sklearn.model_selection import train_test_split
import gc

In [5]:
gc.disable()
train_data_path = 'data/processed/train.pkl'
data = pickle.load(open(train_data_path, 'r'))
gc.enable()

In [6]:
logger = init_log('log/sk_model.log')
sk_train_path = 'data/processed/sk_train.pkl'

# Pair level or sentence level features
rebuild = True
features = ['s1_wlen', 's1_clen', 'jaccard_char_unigram', 'jaccard_char_bigram',\
 			'jaccard_char_trigram', 'jaccard_word_unigram',# 'LevenshteinDistance_char',\
 			'LevenshteinDistance_word', 'word_bool', 's1_word_lsa', 's1_char_lsa']
# features = ['s1_word_lsa']
# features = ['s1_char_lsa', 's1_word_lsa']
# logger.info("Loading training data")
if rebuild or not os.path.exists(sk_train_path):
#     logger.info("Processing training data")
    X_features = []
    for feat in features:
        if data[feat][0] == 's':
            X_features.append((data[feat][1]*data[feat.replace('1', '2')][1]).reshape(-1,data[feat][2]))
            X_features.append(np.abs(data[feat][1]-data[feat.replace('1', '2')][1]).reshape(-1,data[feat][2]))
        elif data[feat][0] == 'p':
            if feat == 'word_bool':
                X_features.append(np.squeeze(data[feat][1]))
            elif len(data[feat][1].shape) == 1:
                X_features.append(data[feat][1].reshape(-1,1))
#                 X_features.append(np.power(data[feat][1].reshape(-1,1), 2))
            else:
                X_features.append(np.squeeze(data[feat][1]))
#                 X_features.append(np.power(np.squeeze(data[feat][1]), 2))

#     for f in X_features:
#         print(f.shape)
    X = np.concatenate(X_features, axis=1)
    y = data['label'][1]
#     del data
#     pickle.dump({"X":X, "y":y}, open(sk_train_path, 'w'))
else:
    data = pickle.load(open(sk_train_path, 'r'))
    X = data['X']
    y = data['y']
#     del data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=666)
print("Number of features", X_train.shape[1])


('Number of features', 533)


In [ ]:
configs = [
    {
        'module': 'linear_model',
        'clf': 'LogisticRegression',
        'kwargs': {
            'class_weight': 'balanced', 
            'C':10.,
        }
    }
]
preds = []
for c in configs:
    clf = SKModel(c)
    # print(clf.clf)
    clf.fit(X_train, y_train)
    # score = clf.score(valid_X, valid_y)
    pred = clf.predict(X_test)
    preds.append(pred)
    print(c['clf'])
    print(clf.score(X_test, y_test))
    print()

In [76]:
for i in np.arange(0.4, 0.9, 0.01):
    print(i, clf.score(X_test, y_test, threshold=i))

(0.4, (0.3763045158473518, 0.5161007025257512, 0.24352922021104847, 0.8274336278609327))
(0.41000000000000003, (0.379091373931204, 0.5317135050222762, 0.24742442150862615, 0.810287610171301))
(0.42000000000000004, (0.38207886896916166, 0.5464480873783716, 0.2514875743347064, 0.7948008845161499))
(0.43000000000000005, (0.38328060049806373, 0.5594262294536079, 0.2544893886714149, 0.7759955747920378))
(0.44000000000000006, (0.38685170692205995, 0.5740632317941, 0.2592732065036202, 0.7616150438265403))
(0.45000000000000007, (0.38977873928493617, 0.5875292739473527, 0.26372338342181606, 0.746681415516216))
(0.4600000000000001, (0.3940604831364429, 0.6017759562254317, 0.2693322508079293, 0.733960176585199))
(0.4700000000000001, (0.39553681528439927, 0.614071038191445, 0.2732840548525271, 0.7157079642059136))
(0.4800000000000001, (0.3974050652578424, 0.628317720469524, 0.2783071127236191, 0.6946902651024943))
(0.4900000000000001, (0.4013690004920061, 0.6416861826071735, 0.2845584835218311, 0.